In [26]:
using LinearAlgebra
using SparseArrays
using Arpack
using KrylovKit
using DelimitedFiles

## Now Let's construct State Sectors

The Hamiltonian is:
$H=-t \sum_{\langle i,j \rangle,s}(c_{i,s}^+c_{j,s}+c_{j,s}^+c_{i,s}) + \sum_{i} \mu(n_{i,+}+n_{i,-}) + \sum_{i} U n_{i,+}n_{i,-} $

States with different particle number $N = \sum_{i,j} \mu(n_{i,+}+n_{j,-})$ and total $m_z$ values don't mix. 

For a $N_{site}$ sites cluster, there can be maximally $N = 2 N_{site}$ particles. With $N$ particles, there could be $0\leq m \leq N$ spin-up particles, and $N-m$ spin-down particles. In this particular sector, there would be $(_{N_{site}}^m)(_{N_{site}}^{(N-m)})$ states. All sectors should have $4^{N_{site}}$ states in total, which should be checked in the code.

First we need to loop over all states, and construct a list of states for $N=0\to N_{site}$, $m=0\to N$.

- Note that we're using the odd places of the binary representation of a Integer to denote the occupancy of spin-up particles, and even places for spin-down particles. e.g. ($10\; 00\; 01\; 11\;...$) means there's a spin-up particle at the 1st site, a spin down particles at the 3rd site, and the 4th site is doubly occupied...

In [2]:
a=[1,2,3,4,5,6,7,8,9,0]
for (ind, a) in enumerate(a)
    println(ind, " ", a)
end

1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 0


In [3]:
"""
    Check for the state, how many spins points up. 
    Input: state: the state to check.
           N: the number of sites in the cluster.

return the number of spin-up and spin-down particles in the state: (N_up, N_dn)
"""
function chk_up_dn(state::Int64,N::Int64)
    state_binary = digits!(zeros(Int64, 64), state, base = 2)[1:(2*N)]
    up_state_binary = state_binary[1:2:(2*N)]
    dn_state_binary = state_binary[2:2:(2*N)]
    return (sum(up_state_binary), sum(dn_state_binary))
end

"""
    Loop over all states, check to which sector the state belongs.
return a dictionary:
    :states::Array{Array{Int,1},2} : states[N_up][N_dn] gives an array of states in the (N_up, N_dn) sector
    :state_tot::Array{Int,2} : state_tot[N_up][N_dn] gives the total number of states in the (N_up, N_dn) sector.
    :state_num::Dict{Int64, Int64} : give the state as key, returns the numbering of the state in its (N_up, N_dn) sector.
"""
function sectors_info_gen(;N::Int64)
    states::Array{Array{Int,1},2} = Array{Int,1}[[] for N_up=0:N, N_dn=0:N]
    state_tot::Array{Int,2} = Int[0 for N_up=0:N, N_dn=0:N]
    state_num::Dict{Int64, Int64} = Dict{Int64, Int64}()
    for state in 0:(4^N-1)
        (N_up,N_dn) = chk_up_dn(state, N)
        push!(states[N_up+1, N_dn+1], state)
        state_tot[N_up+1, N_dn+1] += 1
        state_num[state] = state_tot[N_up+1, N_dn+1]
    end
    @assert(sum(state_tot) == 4^N, "total number of states is not 4^N")
    return Dict{Symbol, Any}(:states => states, :state_tot => state_tot, :state_num => state_num)
end

N=5
@time sectors_info = sectors_info_gen(N=N)
sectors_info[:state_tot][2,1]

  0.167324 seconds (549.21 k allocations: 28.464 MiB)


5

## Now we can construct the Hamiltonian for a given cluster in a particular sector with $(N_{up}, N_{dn})$

The Hamiltonian is:
$H=-t \sum_{\langle i,j \rangle,s}(c_{i,s}^+c_{j,s}+c_{j,s}^+c_{i,s}) - \sum_{i} \mu(n_{i,+}+n_{i,-}) + \sum_{i} U n_{i,+}n_{i,-} $

In [4]:
using SparseArrays

function update_val(row_inds, col_inds, vals;row_ind, col_ind, val)
    push!(row_inds, row_ind)
    push!(col_inds, col_ind)
    push!(vals, val)
end

function H_sector(;t::Real, μ::Real, U::Real, N::Int64, N_up::Int64, N_dn::Int64, sectors_info::Dict{Symbol,Any}, bonds)
    row_inds = Int64[]
    col_inds = Int64[]
    vals = Float64[]
    states = sectors_info[:states][N_up+1, N_dn+1]
    state_tot = sectors_info[:state_tot][N_up+1, N_dn+1]
    state_num = sectors_info[:state_num]
    for state in states #loop over all states in the sector
        state_binary = digits!(zeros(Int64, 64), state, base = 2)[1:(2*N)]
        state_binary_up = state_binary[1:2:(2*N)]
        state_binary_dn = state_binary[2:2:(2*N)]
        #==================== Hubbard U term =====================#
        Hu = 0
        for site in 1:N 
            Hu += U*state_binary_up[site]*state_binary_dn[site]
        end
        update_val(row_inds, col_inds, vals, row_ind = state_num[state], col_ind = state_num[state], val = Hu)
        #================= chemical potential term =================#
        update_val(row_inds, col_inds, vals, row_ind = state_num[state], col_ind = state_num[state], val = -μ * (N_up+N_dn))
        #================= Kinetic term ===============#
        for bond in bonds #bond=[s1, s2], where s1,s2 are the two sites of the bond
            s1 = bond[1]; s2 = bond[2]
            #================== spin-up entries ================#
            if (state_binary_up[s1], state_binary_up[s2])==(1,0) || (state_binary_up[s1], state_binary_up[s2])==(0,1)
                sgn = (-1)^sum(state_binary[(2*s1):(2*(s2-1))])
                flipped_state = state ⊻ (1<<((2*s1-1)-1))
                flipped_state = flipped_state ⊻ (1<<((2*s2-1)-1))
                update_val(row_inds, col_inds, vals, row_ind = state_num[state], col_ind = state_num[flipped_state], val = sgn * (-t))
            end
            #================== spin-dn entries ================#
            if (state_binary_dn[s1], state_binary_dn[s2])==(1,0) || (state_binary_dn[s1], state_binary_dn[s2])==(0,1)
                sgn = (-1)^sum(state_binary[(2*s1+1):(2*s2-1)])
                flipped_state = state ⊻ (1<<(2*s1-1))
                flipped_state = flipped_state ⊻ (1<<(2*s2-1))
                update_val(row_inds, col_inds, vals, row_ind = state_num[state], col_ind = state_num[flipped_state], val = sgn * (-t))
            end
        end
    end
    return sparse(row_inds, col_inds, vals, state_tot, state_tot, +)
end
bonds = [[1,2],[2,3],[2,4],[3,5],[4,5]]
H_sector(;t=1, μ=5.7-0.5*6.4, U=6.4, N=5, N_up=1, N_dn=1, sectors_info=sectors_info, bonds=bonds)

25×25 SparseMatrixCSC{Float64,Int64} with 125 stored entries:
  [1 ,  1]  =  1.4
  [2 ,  1]  =  1.0
  [3 ,  1]  =  -1.0
  [1 ,  2]  =  1.0
  [2 ,  2]  =  -5.0
  [4 ,  2]  =  1.0
  [5 ,  2]  =  -1.0
  [10,  2]  =  -1.0
  [1 ,  3]  =  -1.0
  [3 ,  3]  =  -5.0
  [4 ,  3]  =  -1.0
  [7 ,  3]  =  -1.0
  ⋮
  [22, 23]  =  -1.0
  [23, 23]  =  -5.0
  [25, 23]  =  -1.0
  [12, 24]  =  1.0
  [16, 24]  =  -1.0
  [22, 24]  =  -1.0
  [24, 24]  =  -5.0
  [25, 24]  =  -1.0
  [19, 25]  =  1.0
  [20, 25]  =  1.0
  [23, 25]  =  -1.0
  [24, 25]  =  -1.0
  [25, 25]  =  1.4

In [5]:
H = H_sector(;t=1, μ=8.9, U=6.4, N=5, N_up=5, N_dn=5, sectors_info=sectors_info, bonds=bonds)
(Es, states) = eigen(Matrix(H))

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
1-element Array{Float64,1}:
 -57.0
eigenvectors:
1×1 Array{Float64,2}:
 1.0

## Calculate Properties for the system
Need to loop over all sectors($m=0$-$N$). Diagonalize the sector, loop over all eigen vectors. For each eigen vector,  Calculate contribution to the

1. "denominator"(partition fcn) $P=e^{-\beta E}$
2. "Energy" $=EP$
3. "Esq" $=E^2P$
4. "Magnetization" $=MP$
4. "Msq" $=M^2P$

In [6]:
function measure(;T::Real, t::Real, μ::Real, U::Real, N::Int64, sectors_info::Dict{Symbol,Any}, bonds::Array{Array{Int,1},1})
    β = 1/T
    P::Float64 = 0
    E::Float64 = 0
    Esq::Float64 = 0
    M::Float64 = 0
    Msq::Float64 = 0
    N_tot::Float64 = 0
    for N_up=0:N, N_dn=0:N
        #println("N_up=$N_up, N_dn=$N_dn")
        H_m = H_sector(;t=t, μ=μ, U=U, N=N, N_up=N_up, N_dn=N_dn, sectors_info=sectors_info, bonds=bonds)
        (Es, states) = eigen(Hermitian(Matrix(H_m)))
        for (n, En) in enumerate(Es)
            ΔP = exp(-β*En)
            P += ΔP
            E += En * ΔP
            Esq += En^2 * ΔP
            M += ((N_up-N_dn)*1/2) * ΔP
            Msq += ((N_up-N_dn)*1/2)^2 * ΔP
            N_tot += (N_up+N_dn) * ΔP
        end
    end
    return (P,E,Esq,M,Msq,N_tot)
end

@time result = measure(;T=0.10025884, t=1, μ=5.7+3.2, U=6.4, N=5, sectors_info=sectors_info, bonds=bonds)
N_tot=result[6]/result[1]
E_tot=result[2]/result[1]
N_tot
#E_tot+5.7*N_tot

  0.103342 seconds (268.21 k allocations: 17.273 MiB)


9.949770629608272

In [28]:
Ts = readdlm("../data/Properties.dat")[2:end,2] #temperature of the data file of Prof.Khatami
open("../data/properties_Yutan.dat", "w") do io
    writedlm(io, ["T" "<N>" "<E>"])
    for T in Ts
        result = measure(;T=T, t=1, μ=5.7+3.2, U=6.4, N=5, sectors_info=sectors_info, bonds=bonds)
        N_tot=result[6]/result[1]
        E_tot=result[2]/result[1]
        E_tot = E_tot+5.7*N_tot+5/4*6.4
        writedlm(io, [T N_tot E_tot])
    end
end;


In [8]:
# Test code
rows=[1,1,2,2]
cols=[1,2,1,2]
vals = [0,0.5*im,-0.5*im, 0]
H=sparse(rows, cols, vals, 2, 2, +)
eigs1 = eigsolve(H, 2, :SR, eltype(H), tol = 10^(-20))
typeof(H)

SparseMatrixCSC{Complex{Float64},Int64}

70-element Array{Any,1}:
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 ⋮  
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7
 5.7